In [1]:
import os
os.environ['LOGS_DIRECTORY'] = '../eval_logs'

import sys
sys.path.insert(0, '..')

In [2]:
import json
import random

from tqdm.auto import tqdm
from pydantic import BaseModel
from pydantic_ai import Agent

from main import initialize_index, initialize_agent
from logs import log_interaction_to_file, LOG_DIR

In [3]:
LOG_DIR.absolute()

WindowsPath('C:/Users/alexe/git/aihero/code/eval/../eval_logs')

In [4]:
index = initialize_index()
agent = initialize_agent(index)

Starting AI FAQ Assistant for DataTalksClub/faq
Initializing data ingestion...
Data indexing completed successfully!
Initializing search agent...
Agent initialized successfully!


In [5]:
question_generation_prompt = """
You are helping to create test questions for an AI agent that answers questions about a data engineering course.

Based on the provided FAQ content, generate realistic questions that students might ask.

The questions should:

- Be natural and varied in style
- Range from simple to complex
- Include both specific technical questions and general course questions

Generate one question for each record.
""".strip()

class QuestionsList(BaseModel):
    questions: list[str]

question_generator = Agent(
    name="question_generator",
    instructions=question_generation_prompt,
    model='gpt-4o-mini',
    output_type=QuestionsList
)

In [6]:
sample = random.sample(index.docs, 10)
prompt = [d['content'] for d in sample]

In [9]:
result = await question_generator.run(user_prompt=json.dumps(prompt))
question_list = result.output

In [10]:
for question in tqdm(question_list.questions):
    print(question)
    result = await agent.run(user_prompt=question)
    print(result.output)
    log_interaction_to_file(agent, result.new_messages(), source='ai-generated')
    print()

  0%|          | 0/10 [00:00<?, ?it/s]

How can I fix the error that says `No matching signature for operator '=' for argument types: 'STRING', 'INT64'` in my SQL code?
The error message `No matching signature for operator '=' for argument types: 'STRING', 'INT64'` typically occurs when you attempt to compare or assign a string type with an integer type. Here are some solutions to fix this issue:

1. **Ensure Data Type Consistency**: Make sure the fields you are comparing or assigning are of the same data type. For example, if one field is `STRING`, the other should also be converted to `STRING`.

   If you are using a `CASE` statement or a comparison in an SQL query, ensure that number literals are enclosed in quotes so they are treated as strings. For instance:
   ```sql
   CASE payment_type
   WHEN '1' THEN 'Credit card'
   WHEN '2' THEN 'Cash'
   ...
   END
   ```

2. **Casting Fields**: If one field is an integer (like `INT64`) and the other is a string, you can convert the integer to a string using a cast:
   ```sql
  